## Importing libraries

In [1]:
import spacy
import pandas as pd
from tqdm import tqdm as tq

## loading the dataset from kaggle

In [2]:
# Load the dataset to see its structure
url_data = 'https://raw.githubusercontent.com/TeodorRusKvi/Tekstanalyse/main/git_NLP_data/'

df = pd.read_csv(url_data+ 'file_name.csv')
df.head()

,Title,Political Lean,Score,Id,Subreddit,URL,Num of Comments,Text,Date Created
0,"No matter who someone is, how they look like, ...",Liberal,1,t5fybt,socialism,https://v.redd.it/ng5fyl7hp2l81,0,NaN,1.646272e+09
1,Biden speech draws 38.2 million U.S. TV viewers,Liberal,6,t5fqdn,democrats,https://www.reuters.com/world/us/biden-speech-...,1,NaN,1.646271e+09
2,State of the union,Liberal,1,t5fj9a,DemocraticSocialism,https://www.reddit.com/r/DemocraticSocialism/c...,1,Who watched the state of the union last night ...,1.646270e+09
3,We Should Just Give Poor People Money,Liberal,7,t5f7n9,SocialDemocracy,https://youtu.be/a80kRjpubG0,3,NaN,1.646270e+09
4,Do it for the Dew,Liberal,6,t5es2c,democrats,https://i.redd.it/drmunn90f2l81.jpg,1,NaN,1.646268e+09


## Exploring the data

In [3]:
# Analyze the 'Subreddit' column
subreddit_counts = df['Subreddit'].value_counts()

# Number of unique subreddits
unique_subreddits = len(subreddit_counts)

# Display the number of unique subreddits and the first few rows of the distribution
print(f'Unique Subreddits: {unique_subreddits}')
subreddit_counts

Unique Subreddits: 15


Subreddit
conservatives          1000
SocialDemocracy         997
alltheleft              997
socialism               975
Libertarian             975
Capitalism              975
progressive             974
republicans             948
democrats               941
feminisms               935
DemocraticSocialism     922
Liberal                 904
anarchocapitalism       637
Communist               574
RadicalFeminism         100
Name: count, dtype: int64

In [4]:
political_lean_counts = df['Political Lean'].value_counts()
political_lean_counts

Political Lean
Liberal         8319
Conservative    4535
Name: count, dtype: int64

### Combining the text and title column

In [5]:
# Convert the 'Text' column to string type
df['Text'] = df['Text'].astype(str)

# Concatenate the 'Title' and 'Text' columns
#This is because we want as much text as possible
df['All_text'] = df['Title'] + ' ' + df['Text']

### Checking for empty text rows in the column

In [6]:
cols_to_check = ['All_text']  # Sett inn de relevante kolonnenavnene
empty_rows = df[cols_to_check].isna().all(axis=1)
len(empty_rows==False)

12854

### Preprocessing with spacy

In [7]:
tq.pandas(desc='Processing')
# Load English tokenizer, tagger, parser, NER and word vectors
nlp = spacy.load("en_core_web_sm")

# Define a list of stopwords (this is just an example, ensure to load or define a comprehensive list)
stop_words = spacy.lang.en.stop_words.STOP_WORDS

# Function to tokenize and lemmatize a single text, while removing stopwords and keeping certain abbreviations
def tokenize_and_lemmatize_text(text):
    doc = nlp(text)  # Process the text with spaCy
    lemmatized_tokens = []
    for token in (doc):
        # Check if the token matches specific abbreviations or consists of alphabetic characters
        if token.text in ['U.S.', 'U.S.A.'] or (token.is_alpha and token.lemma_.lower() not in stop_words):
            # Directly add the abbreviations or lemmatize and lower-case other tokens
            token_to_add = token.text if token.text in ['U.S.', 'U.S.A.'] else token.lemma_.lower()
            lemmatized_tokens.append(token_to_add)
    return ' '.join(lemmatized_tokens)


# Apply the tokenization and lemmatization function to each row in the column
df['Processed'] = df['All_text'].progress_apply(tokenize_and_lemmatize_text)
#Fixing US and USA to words instead of abbreviation
df['Processed'] = df['Processed'].replace(['U.S.', 'U.S.A.'], ['US', 'USA'], regex=True)
df['All_text'] = df['All_text'].replace(['U.S.', 'U.S.A.'], ['US', 'USA'], regex=True)
df['Processed'] = df['Processed'].astype(str)
df['All_text'] = df['All_text'].astype(str)

Processing: 100%|██████████| 12854/12854 [04:40<00:00, 45.77it/s]


### Creating columns with the POS tags and Dependency tags

In [8]:
def extract_text_features(text):
    doc = nlp(text)
    dependency_tags = [token.dep_ for token in doc]
    pos_tags = [token.pos_ for token in doc]
    
    # Extract named entities, ensure to join multiple-word entities, and represent them with their labels
    named_entities = [f"{ent.text} ({ent.label_})" for ent in doc.ents]
    
    return dependency_tags, pos_tags, named_entities

# Apply the function to the 'Processed' column and create new columns for each feature
df['Dependency_Tags'], df['POS_Tags'], df['Named_Entities'] = zip(*df['Processed'].apply(extract_text_features))

# Show the first few rows to verify the new columns

## A look at the new dataframe 

### Creating a new dataset for further analyzis

In [10]:
df.rename(columns={'Political Lean': 'Political_Lean'}, inplace=True)
# Create a new dataframe with only the 'All_text', 'Political Lean', and 'Subreddit' columns
new_df = df[['All_text', 'Processed', 'Political_Lean', 'Subreddit', 'Dependency_Tags', 'POS_Tags', ]].copy()
X_text = new_df['Processed']
#Saving the dataframe
new_df.to_csv('new_df.csv', index=False)

In [11]:
new_df.head()

,All_text,Processed,Political_Lean,Subreddit,Dependency_Tags,POS_Tags
0,"No matter who someone is, how they look like, ...",matter look like language speak wear remember ...,Liberal,socialism,"[advmod, ROOT, prep, compound, compound, pobj,...","[ADV, VERB, ADP, NOUN, NOUN, NOUN, VERB, ADJ, ..."
1,Biden speech draws 38.2 million US TV viewers nan,biden speech draw million US tv viewer nan,Liberal,democrats,"[compound, nsubj, ROOT, nummod, compound, comp...","[PROPN, NOUN, VERB, NUM, PROPN, PROPN, NOUN, P..."
2,State of the union Who watched the state of th...,state union watch state union night opinion,Liberal,DemocraticSocialism,"[compound, nsubj, ROOT, compound, compound, co...","[NOUN, PROPN, VERB, NOUN, PROPN, NOUN, NOUN]"
3,We Should Just Give Poor People Money nan,poor people money nan,Liberal,SocialDemocracy,"[amod, nsubj, compound, ROOT]","[ADJ, NOUN, PROPN, PROPN]"
4,Do it for the Dew nan,dew nan,Liberal,democrats,"[compound, ROOT]","[PROPN, PROPN]"
